In [1]:
import nibabel as nb
import numpy as np
import scipy.io
import scipy
from scipy import stats

In [2]:
left_movie_data = scipy.io.loadmat('E:/FPP_files_and_codes/FPP_files/Movie_watching_avg_data/notmean_left_Meanfile.mat')['Left_data']
right_movie_data = scipy.io.loadmat('E:/FPP_files_and_codes/FPP_files/Movie_watching_avg_data/notmean_right_Meanfile.mat')['Right_data']
FPPs_right = (nb.load('E:/FPP_files_and_codes/FPP_files/FPPs_combined/FPPs.rh.func.gii').darrays[0].data).T
FPPs_left = (nb.load('E:/FPP_files_and_codes/FPP_files/FPPs_combined/FPPs.lh.func.gii').darrays[0].data).T
right_FPPs_indices = [np.nonzero(FPPs_right[i]) for i in range(5)]
left_FPPs_indices = [np.nonzero(FPPs_left[i]) for i in range(5)]
activity_of_FPPs = [np.concatenate((right_movie_data[right_FPPs_indices[i]], left_movie_data[left_FPPs_indices[i]])) for i in range(5)]
mean_FPPs_activity = np.array([np.mean(activity_of_FPPs[i], axis=0) for i in range(5)])

# Right

In [8]:
right_FPPs_indices

[(array([ 5107,  5108,  5109,  5110,  5111,  5112,  5113,  5114,  6177,
          6207,  6208,  6209,  6236,  6237,  6238,  6239,  6264,  6265,
          6266,  6267,  6268,  6269,  6291,  6292,  6293,  6294,  6295,
          6296,  6297,  6318,  6319,  6320,  6321,  6322,  6323,  6324,
          6344,  6345,  6346,  6347,  6348,  6349,  6350,  6369,  6370,
          6371,  6372,  6373,  6374,  6375,  6394,  6395,  6396,  6397,
          6398,  6399,  6420,  6421,  6422,  6444, 30352, 30353, 30354,
         30355, 30386, 30387, 30388, 30389, 30390, 30419, 30420, 30421,
         30422, 30423, 30451, 30452, 30453, 30454, 30455, 30483, 30484,
         30485, 30486, 30514, 30515, 30516, 30544, 30545, 30573],
        dtype=int64),),
 (array([ 5092,  5093,  5094,  5095,  5096,  5097,  5098,  5099,  5682,
          5683,  5725,  5726,  5727,  5728,  5729,  5768,  5769,  5770,
          5771,  5772,  5810,  5811,  5812,  5813,  5814,  5851,  5852,
          5853,  5854,  5855,  5891,  5892,  5

In [3]:
correlation_right_vertices_FPPs = np.zeros((32492, 5))
pvalue_right_vertices_FPPs = np.zeros((32492, 5))

for i in range(right_movie_data.shape[0]):
    for j in range(5):
        corr,p_value = stats.pearsonr(right_movie_data[i], mean_FPPs_activity[j])
        correlation_right_vertices_FPPs[i,j] = corr
        pvalue_right_vertices_FPPs[i,j] = p_value

C:\Users\roza\AppData\Local\Temp\ipykernel_23372\3608627345.py:6: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr,p_value = stats.pearsonr(right_movie_data[i], mean_FPPs_activity[j])


In [4]:
correlation_right_vertices_FPPs = np.nan_to_num(correlation_right_vertices_FPPs)
pvalue_right_vertices_FPPs = np.nan_to_num(pvalue_right_vertices_FPPs)
corr_right_32 = correlation_right_vertices_FPPs.astype(np.float32)
pvalue_right_32 = pvalue_right_vertices_FPPs.astype(np.float32)

In [5]:
corr_right_32.shape

(32492, 5)

In [6]:
temp_r_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.R.func.gii'
temp_r = nb.load(temp_r_path)
for i in range(1,6):
    temp_r.darrays[0].data = corr_right_32[:,i-1]
    temp_r.to_filename(f'E:/FPP_files_and_codes/FPP_files/correlation_map_FPPs/correlation_FPP{i}_with_right_vertices.rh.func.gii')

In [10]:
sorted_correlation_right_vertices_FPPs = np.sort(correlation_right_vertices_FPPs, axis=1)[:, ::-1]
sorted_pvalue_right_vertices_FPPs = np.sort(pvalue_right_vertices_FPPs, axis=1)[:, ::-1]
sorted_prefered_FPP_each_vertices = np.argsort(correlation_right_vertices_FPPs, axis=1)[:, ::-1]

In [11]:
preferred_FPP_each_vertices = sorted_prefered_FPP_each_vertices[:,0] + 1

In [12]:
preferred_FPP_each_vertices

array([5, 5, 5, ..., 5, 5, 5], dtype=int64)

In [13]:
positive_preferred_corr_each_vertices_right = sorted_correlation_right_vertices_FPPs[:,0][sorted_correlation_right_vertices_FPPs[:,0]> 0]

In [14]:
mean_sorted_correlation_right_vertices_FPPs_right = np.mean(positive_preferred_corr_each_vertices_right) 

In [15]:
mean_sorted_correlation_right_vertices_FPPs_right

0.3510572207193558

In [16]:
threshold = 0.3
above_threshold_vertices = sorted_correlation_right_vertices_FPPs[:,0] >= threshold

In [17]:
above_threshold_vertices

array([False, False, False, ..., False, False,  True])

In [18]:
preferred_FPP_above_threshold_vertices = np.zeros_like(preferred_FPP_each_vertices)

preferred_FPP_above_threshold_vertices[above_threshold_vertices] = preferred_FPP_each_vertices[above_threshold_vertices]

In [19]:
preferred_FPP_above_threshold_vertices

array([0, 0, 0, ..., 0, 0, 5], dtype=int64)

In [20]:
temp_r_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.R.func.gii'
temp_r = nb.load(temp_r_path)
temp_r.darrays[0].data = preferred_FPP_above_threshold_vertices
temp_r.to_filename('E:/FPP_files_and_codes/FPP_files/winner_take_all_correlation_maps/theresholded_winner_take_all_FPPs.R.func.gii')

In [31]:
mean_FPPs_activity

array([[ 0.11503519,  0.03718783,  0.06390971, ..., -0.09333008,
        -0.04491226, -0.01514777],
       [ 0.08077597, -0.02147286,  0.03186865, ..., -0.16496121,
        -0.13001685, -0.07163416],
       [-0.04819481, -0.19603622, -0.02925085, ..., -0.48429933,
        -0.47145943, -0.40271844],
       [-0.12274933, -0.24787016, -0.14828766, ..., -0.43621268,
        -0.43554419, -0.40134424],
       [-0.31982471, -0.23964374, -0.09059742, ..., -0.37779491,
        -0.33173222, -0.25312564]])

# Left

In [7]:
correlation_left_vertices_FPPs = np.zeros((32492, 5))
pvalue_left_vertices_FPPs = np.zeros((32492, 5))

for i in range(left_movie_data.shape[0]):
    for j in range(5):
        corr,p_value = stats.pearsonr(left_movie_data[i], mean_FPPs_activity[j])
        correlation_left_vertices_FPPs[i,j] = corr
        pvalue_left_vertices_FPPs[i,j] = p_value

C:\Users\roza\AppData\Local\Temp\ipykernel_31312\4226043883.py:6: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr,p_value = stats.pearsonr(left_movie_data[i], mean_FPPs_activity[j])


In [8]:
correlation_left_vertices_FPPs = np.nan_to_num(correlation_left_vertices_FPPs)
pvalue_left_vertices_FPPs = np.nan_to_num(pvalue_left_vertices_FPPs)
corr_left_32 = correlation_left_vertices_FPPs.astype(np.float32)
pvalue_left_32 = pvalue_left_vertices_FPPs.astype(np.float32)

In [21]:
temp_l_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.L.func.gii'
temp_l = nb.load(temp_l_path)
for i in range(1,6):
    temp_l.darrays[0].data = corr_left_32[:,i-1]
    temp_l.to_filename(f'E:/FPP_files_and_codes/FPP_files/correlation_map_FPPs/correlation_FPP{i}_with_left_vertices.lh.func.gii')

In [22]:
np.nan_to_num(correlation_left_vertices_FPPs, copy=False, nan=0)
np.nan_to_num(pvalue_left_vertices_FPPs, copy=False, nan=0)

array([[3.00534071e-213, 2.81098395e-245, 3.67182745e-193,
        3.52345891e-099, 1.04126060e-006],
       [2.93171881e-071, 1.30804231e-311, 1.35168820e-289,
        3.98668146e-198, 3.00392521e-047],
       [1.76571062e-087, 3.10963046e-052, 2.57022263e-033,
        6.22037987e-036, 1.70389417e-008],
       ...,
       [1.87987857e-067, 2.13922175e-039, 9.66328952e-001,
        4.99072668e-056, 1.39527882e-177],
       [2.26174587e-076, 1.81237928e-056, 1.72982849e-003,
        6.06945606e-032, 1.17762806e-119],
       [1.49592883e-039, 4.82013888e-035, 5.74828667e-001,
        6.84606903e-056, 1.14026189e-157]])

In [23]:
sorted_correlation_left_vertices_FPPs = np.sort(correlation_left_vertices_FPPs, axis=1)[:, ::-1]
sorted_pvalue_left_vertices_FPPs = np.sort(pvalue_left_vertices_FPPs, axis=1)[:, ::-1]
sorted_prefered_FPP_each_vertices_left = np.argsort(correlation_left_vertices_FPPs, axis=1)[:, ::-1]

In [24]:
preferred_FPP_each_vertices_left = sorted_prefered_FPP_each_vertices_left[:,0] + 1

In [25]:
positive_preferred_corr_each_vertices_left = sorted_correlation_left_vertices_FPPs[:,0][sorted_correlation_left_vertices_FPPs[:,0]>0]

In [26]:
mean_sorted_correlation_left_vertices_FPPs= np.mean(positive_preferred_corr_each_vertices_left) 

In [27]:
threshold = 0.3
above_threshold_vertices_left = sorted_correlation_left_vertices_FPPs[:,0] >= threshold

In [28]:
preferred_FPP_above_threshold_vertices_left = np.zeros_like(preferred_FPP_each_vertices_left)

preferred_FPP_above_threshold_vertices_left[above_threshold_vertices_left] = preferred_FPP_each_vertices_left[above_threshold_vertices_left]

In [29]:
temp_l_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.L.func.gii'
temp_l = nb.load(temp_l_path)
temp_l.darrays[0].data = preferred_FPP_above_threshold_vertices_left
temp_l.to_filename('E:/FPP_files_and_codes/FPP_files/winner_take_all_correlation_maps/theresholded_winner_take_all_FPPs.L.func.gii')